In [1]:
!hostname

5736089a228b


In [2]:
!jupyter --version

4.4.0


https://stackoverflow.com/questions/276052/how-to-get-current-cpu-and-ram-usage-in-python

https://pypi.org/project/psutil/<BR>
https://psutil.readthedocs.io/en/latest/

potential additional work:
* add strace capture
 * https://github.com/blapid/strace2py
 * https://python-ptrace.readthedocs.io/en/latest/syscall.html
 * https://github.com/dirtyharrycallahan/pystrace

In [3]:
!pip install psutil

In [4]:
import requests
import json
import subprocess
import psutil
import pickle
import os
import time
import datetime
import pandas
print('pandas',pandas.__version__)
import matplotlib.pyplot as plt

pandas 0.23.4


# one-time system details

In [5]:
print(psutil.cpu_freq())

None


In [6]:
pid = os.getpid()
print(pid)

872


In [7]:
py = psutil.Process(pid)
print(py)

psutil.Process(pid=872, name='python', started='18:12:46')


In [8]:
py.name()

'python'

In [9]:
py.exe()

'/opt/conda/bin/python3.6'

In [10]:
py.cwd()

'/home/jovyan/jupyter_notebooks/week_09_timeseries/host_streaming_stats'

In [11]:
py.cmdline()

['/opt/conda/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/home/jovyan/.local/share/jupyter/runtime/kernel-47b6945c-36d1-4fe0-a9b2-51a9ef3935ea.json']

# system performance

In [12]:
# gives a single float value
psutil.cpu_percent(percpu=True)

[28.1, 91.6]

In [13]:
psutil.cpu_times_percent(percpu=True)

[scputimes(user=8.3, nice=0.0, system=19.8, idle=71.9, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0),
 scputimes(user=62.4, nice=0.0, system=29.4, idle=8.1, iowait=0.0, irq=0.0, softirq=0.0, steal=0.0, guest=0.0, guest_nice=0.0)]

In [14]:
for reslt in psutil.cpu_times_percent(percpu=True):
    print(dict(reslt._asdict()))

{'user': 1.0, 'nice': 0.0, 'system': 3.0, 'idle': 1.0, 'iowait': 0.0, 'irq': 0.0, 'softirq': 0.0, 'steal': 0.0, 'guest': 0.0, 'guest_nice': 0.0}
{'user': 2.0, 'nice': 0.0, 'system': 2.0, 'idle': 0.0, 'iowait': 0.0, 'irq': 0.0, 'softirq': 1.0, 'steal': 0.0, 'guest': 0.0, 'guest_nice': 0.0}


In [15]:
# gives an object with many fields
psutil.virtual_memory()

svmem(total=2096164864, available=1243967488, percent=40.7, used=686940160, free=614150144, active=936108032, inactive=432259072, buffers=93143040, cached=701931520, shared=524288, slab=83623936)

In [16]:
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

{'total': 2096164864,
 'available': 1243967488,
 'percent': 40.7,
 'used': 686940160,
 'free': 614150144,
 'active': 936116224,
 'inactive': 432259072,
 'buffers': 93143040,
 'cached': 701931520,
 'shared': 524288,
 'slab': 83623936}

In [17]:
dict(psutil.swap_memory()._asdict())

{'total': 1073737728,
 'used': 17203200,
 'free': 1056534528,
 'percent': 1.6,
 'sin': 86016,
 'sout': 17317888}

In [18]:
dict(psutil.cpu_stats()._asdict())

{'ctx_switches': 43824859,
 'interrupts': 20444126,
 'soft_interrupts': 10073553,
 'syscalls': 0}

In [19]:
psutil.disk_io_counters(perdisk=True).keys()

dict_keys(['loop0', 'loop1', 'loop2', 'loop3', 'loop4', 'loop5', 'loop6', 'loop7', 'nbd0', 'nbd1', 'nbd2', 'nbd3', 'nbd4', 'nbd5', 'nbd6', 'nbd7', 'nbd8', 'nbd9', 'nbd10', 'nbd11', 'nbd12', 'nbd13', 'nbd14', 'nbd15', 'sda', 'sda1', 'sr0', 'sr1', 'sr2'])

In [20]:
dict(psutil.disk_io_counters(perdisk=True)['loop0']._asdict())

{'read_count': 0,
 'write_count': 0,
 'read_bytes': 0,
 'write_bytes': 0,
 'read_time': 0,
 'write_time': 0,
 'read_merged_count': 0,
 'write_merged_count': 0,
 'busy_time': 0}

In [21]:
psutil.sensors_temperatures()

{}

In [22]:
psutil.sensors_fans()

{}

In [23]:
psutil.sensors_battery()

### put everything into a single row

In [24]:
def row_for_system_performance(row):
    if not isinstance(row,dict):
        print("ERROR: expected a dict as input")
        return None
    cpu_pct_list = psutil.cpu_percent(percpu=True)
    for indx in range(len(cpu_pct_list)):
        row['CPU percent for '+str(indx)]=cpu_pct_list[indx]
    
    row['time in unix seconds']=time.time()
    row['datetime']=datetime.datetime.utcfromtimestamp(row['time in unix seconds'])

    indx=0
    for reslt in psutil.cpu_times_percent(percpu=True):
        indx+=1
        for k,v in dict(reslt._asdict()).items():
            row['CPU '+str(indx)+' times '+k]=v
    for disk_label in psutil.disk_io_counters(perdisk=True).keys():
        for k,v in dict(psutil.disk_io_counters(perdisk=True)['loop0']._asdict()).items():
            row['disk '+disk_label+' '+k]=v
    for k,v in dict(psutil.cpu_stats()._asdict()).items():
        row['CPU stats '+k]=v
    for k,v in dict(psutil.virtual_memory()._asdict()).items():
        row['system virtual memory '+k]=v
    for k,v in dict(psutil.swap_memory()._asdict()).items():
        row['system swap memory '+k]=v
    return row

# performance of current process

In [25]:
os.getpid()

872

In [26]:
type(os.getpid())

int

In [27]:
py = psutil.Process(os.getpid())
print(py)

psutil.Process(pid=872, name='python', started='18:12:46')


In [28]:
dict(py.memory_info()._asdict())

{'rss': 110739456,
 'vms': 854994944,
 'shared': 31952896,
 'text': 2232320,
 'lib': 0,
 'data': 189218816,
 'dirty': 0}

In [29]:
memoryUse = py.memory_info()[0]/2.**30  # memory use in GB...I think
print('memory use:', memoryUse,'GB')

memory use: 0.1031341552734375 GB


In [30]:
dict(py.memory_full_info()._asdict())

{'rss': 110739456,
 'vms': 854994944,
 'shared': 31952896,
 'text': 2232320,
 'lib': 0,
 'data': 189218816,
 'dirty': 0,
 'uss': 79925248,
 'pss': 88809472,
 'swap': 0}

In [31]:
py.cpu_percent()

0.0

In [32]:
dict(py.cpu_times()._asdict())

{'user': 3.63, 'system': 2.17, 'children_user': 3.67, 'children_system': 2.31}

In [33]:
py.cmdline()

['/opt/conda/bin/python',
 '-m',
 'ipykernel_launcher',
 '-f',
 '/home/jovyan/.local/share/jupyter/runtime/kernel-47b6945c-36d1-4fe0-a9b2-51a9ef3935ea.json']

In [34]:
dict(py.num_ctx_switches()._asdict())

{'voluntary': 1169, 'involuntary': 1940}

In [35]:
dict(py.io_counters()._asdict())

{'read_count': 6498,
 'write_count': 1164,
 'read_bytes': 225280,
 'write_bytes': 4096,
 'read_chars': 29500475,
 'write_chars': 903245}

In [36]:
def row_for_performance_per_pid(row,pid):
    if not isinstance(row,dict):
        print("ERROR: expected a dict as input for row")
        return None
    if not isinstance(pid,int):
        print("ERROR: expected a int as input for pid")
        return None
    try:
        mem_dict=dict(psutil.Process(pid).memory_full_info()._asdict())
    except NoSuchProcess:
        return row
    for k,v in mem_dict.items():
        row['process memory: '+k]=v

    try:
        cpu_times_dict=dict(psutil.Process(pid).cpu_times()._asdict())
    except NoSuchProcess:
        return row
    for k,v in cpu_times_dict.items():
        row['process cpu times: '+k]=v

    try:
        context_switches_dict=dict(psutil.Process(pid).num_ctx_switches()._asdict())
    except NoSuchProcess:
        return row        
    for k,v in context_switches_dict.items():
        row['process number context switches: '+k]=v

    try:
        io_counter_dict = dict(psutil.Process(pid).io_counters()._asdict())
    except NoSuchProcess:
        return row
    for k,v in io_counter_dict.items():
        row['process io counters: '+k]=v
    return row

# gather data and create plots

In [37]:
polling_period_in_seconds = 1
monitoring_duration_in_seconds=10

In [38]:
list_of_rows=[]
start_time = time.time()
for indx in range(monitoring_duration_in_seconds):
    row = {}
    row = row_for_system_performance(row)
    row = row_for_performance_per_pid(row,os.getpid())
    list_of_rows.append(row)
    time.sleep(polling_period_in_seconds)

print('elapsed time:',time.time()-start_time,'seconds')

df = pandas.DataFrame(list_of_rows)

elapsed time: 11.003600120544434 seconds


In [39]:
print(df.shape)
df.head()

(10, 328)


,CPU 1 times guest,CPU 1 times guest_nice,CPU 1 times idle,CPU 1 times iowait,CPU 1 times irq,CPU 1 times nice,CPU 1 times softirq,CPU 1 times steal,CPU 1 times system,CPU 1 times user,...,system virtual memory buffers,system virtual memory cached,system virtual memory free,system virtual memory inactive,system virtual memory percent,system virtual memory shared,system virtual memory slab,system virtual memory total,system virtual memory used,time in unix seconds
0,0.0,0.0,12.4,0.0,0.0,0.0,0.9,0.0,52.5,34.1,...,93155328,701956096,613007360,432267264,40.7,524288,83697664,2096164864,688046080,1.552846e+09
1,0.0,0.0,93.7,0.0,0.0,0.0,0.9,0.0,4.5,0.9,...,93155328,701931520,612913152,432267264,40.7,524288,83632128,2096164864,688164864,1.552846e+09
2,0.0,0.0,99.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,93155328,701935616,612913152,432267264,40.7,524288,83623936,2096164864,688160768,1.552846e+09
3,0.0,0.0,15.9,0.0,0.0,0.0,0.0,0.0,3.7,80.4,...,93155328,702066688,612786176,432381952,40.7,524288,83640320,2096164864,688156672,1.552846e+09
4,0.0,0.0,90.7,0.0,0.0,0.0,0.0,0.0,1.9,7.4,...,93155328,702070784,612786176,432398336,40.7,524288,83644416,2096164864,688152576,1.552846e+09


In [40]:
df.to_pickle("single_process_df_this_notebook_"+str(time.time())+".pkl")

#df = pandas.read_pickle("single_process_df_this_notebook_1552783848.2168732.pkl")

In [41]:
def scatter_plot_column(df,col_name):
    if not isinstance(df,pandas.core.frame.DataFrame):
        print('ERROR: expected input of type dataframe')
        return None
    plt.plot_date(x=df['datetime'],y=df[col_name])
    plt.xticks(rotation=40)
    _=plt.ylabel(col_name,fontsize=14)
    _=plt.xlabel('time',fontsize=14)
    plt.show()
    return

# track other notebooks

Plan: associate pid with each notebook kernel ID, then track the stats for that notebook by pid

works even when the kernel of the other notebook is restarted

detects when new newbook kernels being added

### get a list of all active sessions

In [42]:
r = requests.get('http://localhost:8888/api/sessions')
r.status_code

200

In [43]:
list_of_kernel_dicts = r.json()
list_of_kernel_dicts[0]

{'id': '166b6493-15f6-473f-a5f3-f4c450c90cc3',
 'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/host_cpu_and_memory.ipynb',
 'name': 'host_cpu_and_memory.ipynb',
 'type': 'notebook',
 'kernel': {'id': '47b6945c-36d1-4fe0-a9b2-51a9ef3935ea',
  'name': 'python3',
  'last_activity': '2019-03-17T18:13:14.361708Z',
  'execution_state': 'idle',
  'connections': 1},
 'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/host_cpu_and_memory.ipynb',
  'name': 'host_cpu_and_memory.ipynb'}}

### from json returned from API, we can extract the notebook kernel IDs

In [44]:
print('notebook kernel IDs:')
for notebook_indx in range(len(list_of_kernel_dicts)):
    print(list_of_kernel_dicts[notebook_indx]['kernel']['id'])

notebook kernel IDs:
47b6945c-36d1-4fe0-a9b2-51a9ef3935ea
ed6bf0d4-9d65-4606-b9e0-07290655f41a
47a170e2-d865-4b15-a89a-40f6ef2e2adb
de8fa631-301b-47a2-be21-5c2ee0bf6952


### get a list of all pids

In [45]:
print('all processes running:')
for p in psutil.process_iter():
    print(p)

all processes running:
psutil.Process(pid=1, name='tini', started='10:54:27')
psutil.Process(pid=6, name='jupyter-lab', started='10:54:28')
psutil.Process(pid=87, name='python', started='11:41:57')
psutil.Process(pid=103, name='bash', started='11:47:10')
psutil.Process(pid=834, name='python', started='17:44:29')
psutil.Process(pid=872, name='python', started='18:12:46')
psutil.Process(pid=900, name='python', started='18:13:06')


### get pid for each notebook

In [46]:
def get_all_kernel_pids():
    dict_of_jupyter_kernel_pids = {}
    for p in psutil.process_iter():
        list_of_commands_per_process = psutil.Process(p.pid).cmdline()
#        print(list_of_commands_per_process)
        if ('ipykernel_launcher' in list_of_commands_per_process):
            dict_of_jupyter_kernel_pids[p.pid] = list_of_commands_per_process[len(list_of_commands_per_process)-1]
        
    return dict_of_jupyter_kernel_pids

In [47]:
dict_of_jupyter_kernel_pids = get_all_kernel_pids()
dict_of_jupyter_kernel_pids

{87: '/home/jovyan/.local/share/jupyter/runtime/kernel-ed6bf0d4-9d65-4606-b9e0-07290655f41a.json',
 834: '/home/jovyan/.local/share/jupyter/runtime/kernel-de8fa631-301b-47a2-be21-5c2ee0bf6952.json',
 872: '/home/jovyan/.local/share/jupyter/runtime/kernel-47b6945c-36d1-4fe0-a9b2-51a9ef3935ea.json',
 900: '/home/jovyan/.local/share/jupyter/runtime/kernel-47a170e2-d865-4b15-a89a-40f6ef2e2adb.json'}

### (alternative method) use `ps -ax` to get the relevant pid for notebooks

not in use

### add pid to dict containing kernel information

In [48]:
def add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts):
    for pid,kernel_path in dict_of_jupyter_kernel_pids.items():
        for notebook_indx in range(len(list_of_kernel_dicts)):
            if (list_of_kernel_dicts[notebook_indx]['kernel']['id'] in kernel_path):
#                print(pid,'is',list_of_kernel_dicts[notebook_indx],'\n')
                list_of_kernel_dicts[notebook_indx]['pid']=pid
    return list_of_kernel_dicts

In [49]:
list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)

In [50]:
list_of_kernel_dicts

[{'id': '166b6493-15f6-473f-a5f3-f4c450c90cc3',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/host_cpu_and_memory.ipynb',
  'name': 'host_cpu_and_memory.ipynb',
  'type': 'notebook',
  'kernel': {'id': '47b6945c-36d1-4fe0-a9b2-51a9ef3935ea',
   'name': 'python3',
   'last_activity': '2019-03-17T18:13:14.361708Z',
   'execution_state': 'idle',
   'connections': 1},
  'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/host_cpu_and_memory.ipynb',
   'name': 'host_cpu_and_memory.ipynb'},
  'pid': 872},
 {'id': '853dfc05-e571-49fa-bd39-a25d57faf780',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
  'name': 'realtime_data_matplotlib.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'ed6bf0d4-9d65-4606-b9e0-07290655f41a',
   'name': 'python3',
   'last_activity': '2019-03-17T17:34:38.256378Z',
   'execution_state': 'idle',
   'connections': 2},
  'notebook': {'path': 'jupyter_notebooks/we

### which kernels are not this notebook?

In [51]:
def kernel_dicts_other_notebooks(list_of_kernel_dicts):
    list_of_kernel_dicts_for_other_notebooks=[]
    for kernel_dict in list_of_kernel_dicts:
        if (str(os.getpid())!=str(kernel_dict['pid'])):
            list_of_kernel_dicts_for_other_notebooks.append(kernel_dict)
    return list_of_kernel_dicts_for_other_notebooks

In [52]:
list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)
list_of_kernel_dicts_for_other_notebooks

[{'id': '853dfc05-e571-49fa-bd39-a25d57faf780',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
  'name': 'realtime_data_matplotlib.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'ed6bf0d4-9d65-4606-b9e0-07290655f41a',
   'name': 'python3',
   'last_activity': '2019-03-17T17:34:38.256378Z',
   'execution_state': 'idle',
   'connections': 2},
  'notebook': {'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/realtime_data_matplotlib.ipynb',
   'name': 'realtime_data_matplotlib.ipynb'},
  'pid': 87},
 {'id': '18960a5f-a2c6-42a6-9e66-fa685b5e1ff0',
  'path': 'jupyter_notebooks/week_09_timeseries/host_streaming_stats/analysis_of_static_timeseries.ipynb',
  'name': 'analysis_of_static_timeseries.ipynb',
  'type': 'notebook',
  'kernel': {'id': '47a170e2-d865-4b15-a89a-40f6ef2e2adb',
   'name': 'python3',
   'last_activity': '2019-03-17T18:13:14.375721Z',
   'execution_state': 'busy',
   'connections': 1},
  'notebook': {

In [53]:
def get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks,
                          url_for_api_sessions='http://localhost:8888/api/sessions'):
    list_of_kernel_dicts = requests.get(url_for_api_sessions).json()
    dict_of_jupyter_kernel_pids = get_all_kernel_pids()
    list_of_kernel_dicts= add_pid_to_kernel_dict(dict_of_jupyter_kernel_pids,list_of_kernel_dicts)
    current_list_of_kernel_dicts_for_other_notebooks = kernel_dicts_other_notebooks(list_of_kernel_dicts)

    list_of_pids_in_previous=[]
    for old_dict in list_of_kernel_dicts_for_other_notebooks:
        list_of_pids_in_previous.append(old_dict['pid'])
        for curr_dict in current_list_of_kernel_dicts_for_other_notebooks:
            if (curr_dict['pid'] not in list_of_pids_in_previous):
                list_of_kernel_dicts_for_other_notebooks.append(curr_dict)
    return list_of_kernel_dicts_for_other_notebooks

# collect data for other notebook and create plots

In [54]:
monitoring_duration_in_seconds=60*10

In [55]:
nb_dict={}
list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids([])

start_time = time.time()
for indx in range(monitoring_duration_in_seconds):
    row = {}
    system_row = row_for_system_performance(row)

    # detect new notebook kernels
    list_of_kernel_dicts_for_other_notebooks = get_other_kernel_pids(list_of_kernel_dicts_for_other_notebooks)
    
    for other_nb in list_of_kernel_dicts_for_other_notebooks:
        row = row_for_performance_per_pid(system_row,int(other_nb['pid']))
        try:
            nb_dict[other_nb['pid']].append(row)
        except KeyError: # new pid for new kernel
            nb_dict[other_nb['pid']] = []
            nb_dict[other_nb['pid']].append(row)
    time.sleep(polling_period_in_seconds)

print('elapsed time:',time.time()-start_time,'seconds')

nb_df = {}
for other_nb in list_of_kernel_dicts_for_other_notebooks:
    nb_df[other_nb['pid']] = pandas.DataFrame(nb_dict[other_nb['pid']])

elapsed time: 623.4142644405365 seconds


In [56]:
nb_df.keys()

dict_keys([])

In [ ]:
for indx in range(len(list_of_kernel_dicts_for_other_notebooks)):
    nb_pid = list_of_kernel_dicts_for_other_notebooks[indx]['pid']
    list_of_kernel_dicts_for_other_notebooks[indx]['df']=nb_df[nb_pid]

In [ ]:
with open("single_process_df_other_notebook_pid_"+str(this_pid)+"_"+str(time.time())+".pkl", 'wb') as handle:
    pickle.dump(list_of_kernel_dicts_for_other_notebooks, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#   list_of_kernel_dicts_for_other_notebooks = pickle.load(handle)